In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/3. 증가율 데이터 정리끝.csv", encoding='cp949') 
df_data = pd.read_csv("../data/4. 이자보상배율 데이터.csv", encoding = 'cp949')

In [2]:
# 공백 제거
df_data.columns = df_data.columns.str.replace("   ","")

# 이자보상배율 계산을 위한 데이터 준비 : 연결+개별로 값 채우기
df_data["[제조]* (정상)영업손익(보고서기재)(IFRS연결)"] = df_data["[제조]* (정상)영업손익(보고서기재)(IFRS연결)"].fillna(df_data["[제조]* (정상)영업손익(보고서기재)(IFRS)"])  
df_data["[제조]이자비용(IFRS연결)"] = df_data["[제조]이자비용(IFRS연결)"].fillna(df_data["[제조]이자비용(IFRS)"])  

# 이자보상배율 계산
df_data["이자보상배율"] = round(df_data["[제조]* (정상)영업손익(보고서기재)(IFRS연결)"]/df_data["[제조]이자비용(IFRS연결)"],2)

# 데이터 분리하기
1. `df_mod_0`: "상장폐지일" 열이 없는 데이터를 선택한 하위 데이터프레임입니다.

2. `df_mod_0_0`: `df_mod_0`에서 "이자보상배율" 열이 결측치, 양의 무한대, 음의 무한대를 포함하지 않는 데이터를 선택한 하위 데이터프레임입니다.

3. `df_mod_1`: "상장폐지일" 열이 있는 데이터를 선택한 하위 데이터프레임입니다.

4. `df_mod_1_1`: `df_mod_1`에서 "이자보상배율" 열이 결측치, 양의 무한대, 음의 무한대를 포함하지 않는 데이터를 선택한 하위 데이터프레임입니다.

In [3]:
# 계산된 이자보상배율 데이터로 새로운 데이터프레임 만들기
df_data_mod = df_data[["회사명","거래소코드","회계년도","이자보상배율"]]

# 무한대 값 확인
df_data_mod[df_data_mod["이자보상배율"]==-np.inf]

# 회계년도를 기준으로 데이터 프레임 합치기
df_data["회계년도"]= pd.to_datetime(df_data["회계년도"],format='%Y/%m')
df["회계년도"]= pd.to_datetime(df["회계년도"])
merged_data = pd.merge(df,df_data, how='left',on=["회사명","거래소코드","회계년도"])

# 필요한 열만 선택
df_mod = merged_data[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '소속코드','이자보상배율','이자보상배율이자비용']]

# "상장폐지일"이 없는 데이터를 선택하여 df_mod_0에 저장
df_mod_0 = df_mod[df_mod["상장폐지일"].isnull()]

# df_mod_0에서 "이자보상배율" 열이 결측치, 양의 무한대, 음의 무한대를 포함하는 데이터 필터링
# (실제로 데이터프레임을 수정하지는 않음)
df_mod_0[df_mod_0["이자보상배율"].isin([np.NaN, np.inf, -np.inf])]

# df_mod_0에서 "이자보상배율" 열이 결측치, 양의 무한대, 음의 무한대를 포함하지 않는 데이터를 선택하여 df_mod_0_0에 저장
df_mod_0_0 = df_mod_0[~df_mod_0["이자보상배율"].isin([np.NaN, np.inf, -np.inf])]

# "상장폐지일"이 없고 "이자보상배율" 열이 결측치, 양의 무한대, 음의 무한대를 포함하지 않는 데이터의 "이자보상배율" 열을 출력
df_mod[df_mod["상장폐지일"].isnull()][df_mod_0["상장폐지일"].isnull()]["이자보상배율"]

# "상장폐지일"이 있는 데이터를 선택하여 df_mod_1에 저장
df_mod_1 = df_mod[~df_mod["상장폐지일"].isnull()]

# df_mod_1에서 "이자보상배율" 열이 결측치, 양의 무한대, 음의 무한대를 포함하는 데이터 필터링
# (실제로 데이터프레임을 수정하지는 않음)
df_mod_1[df_mod_1["이자보상배율"].isin([np.NaN, np.inf, -np.inf])]

# df_mod_1에서 "이자보상배율" 열이 결측치, 양의 무한대, 음의 무한대를 포함하지 않는 데이터를 선택하여 df_mod_1_1에 저장
df_mod_1_1 = df_mod_1[~df_mod_1["이자보상배율"].isin([np.NaN, np.inf, -np.inf])]

# df_mod[~df_mod["상장폐지일"].isnull()]["이자보상배율"]


# 이자보상배율 값 처리
1. "이자보상배율"이 양의 무한대(`np.inf`)이고 "상장폐지일"이 없으면, "이자보상배율"을 해당 열에서 가장 큰 값으로 설정합니다.

2. "이자보상배율"이 음의 무한대(`-np.inf`)이고 "상장폐지일"이 없으면, "이자보상배율"을 해당 열에서 가장 작은 값으로 설정합니다.

3. "이자보상배율"이 결측치이고 "상장폐지일"이 없으면, "이자보상배율"을 해당 열에서 값들의 평균으로 설정합니다.

4. "이자보상배율"이 양의 무한대(`np.inf`)이고 "상장폐지일"이 있으면, "이자보상배율"을 다른 데이터에서 가장 큰 값으로 설정합니다.

5. "이자보상배율"이 음의 무한대(`-np.inf`)이고 "상장폐지일"이 있으면, "이자보상배율"을 다른 데이터에서 가장 작은 값으로 설정합니다.

6. "이자보상배율"이 결측치이고 "상장폐지일"이 있으면, "이자보상배율"을 다른 데이터에서 값들의 평균으로 설정합니다.

이러한 조정 작업을 마치고 나면 "이자보상배율이자비용" 열에 보정된 값을 할당합니다. 이로써 데이터의 누락된 부분을 최대, 최소 또는 평균값으로 채우게 됩니다.

In [4]:

for i in df_mod.index:
    if df_mod.loc[i,"이자보상배율"] == np.inf and df_mod.loc[i,"상장폐지일"] ==np.NaN:
        df_mod.loc[i,"이자보상배율"] = df_mod_0_0["이자보상배율"].max()

    elif df_mod.loc[i,"이자보상배율"] == -np.inf and df_mod.loc[i,"상장폐지일"] ==np.NaN:
        df_mod.loc[i,"이자보상배율"] = df_mod_0_0["이자보상배율"].min()

    elif df_mod.loc[i,"이자보상배율"] == np.NaN and df_mod.loc[i,"상장폐지일"] ==np.NaN:
        df_mod.loc[i,"이자보상배율"] = df_mod_0_0["이자보상배율"].mean()

    elif df_mod.loc[i,"이자보상배율"] == np.inf and df_mod.loc[i,"상장폐지일"] !=np.NaN:
        df_mod.loc[i,"이자보상배율"] = df_mod_1_1["이자보상배율"].max()

    elif df_mod.loc[i,"이자보상배율"] == -np.inf and df_mod.loc[i,"상장폐지일"] !=np.NaN:
        df_mod.loc[i,"이자보상배율"] = df_mod_1_1["이자보상배율"].min()

    elif df_mod.loc[i,"이자보상배율"] == np.NaN and df_mod.loc[i,"상장폐지일"] !=np.NaN:
        df_mod.loc[i,"이자보상배율"] = df_mod_1_1["이자보상배율"].mean()

df["이자보상배율이자비용"] = df_mod["이자보상배율"]

In [5]:
df[df["이자보상배율이자비용"].isin([np.NaN,np.inf,-np.inf])]

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,산업명,매출액증가율,유형자산증가율,재고자산증가율,...,유동자산회전률,당좌자산회전률,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율


In [6]:
df.to_csv("../data/5. 이자보상배율 완료.csv",encoding='cp949',index=False)

In [7]:
print('총기업수 :',len(df["회사명"].unique()))
print('정상기업(중복미포함) :',len(df[df["소속코드"]==5]["회사명"].unique()))
print('상장폐지 기업 :',len(df[df["소속코드"]==4]["회사명"].unique()))

총기업수 : 1399
정상기업(중복미포함) : 1171
상장폐지 기업 : 171
